In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.get_option("display.max_columns")
pd.set_option("display.max_rows", None)

In [2]:
df=pd.read_csv("data.csv")
df.head(2)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1


# Plan :
- Parse salary from "Salary Estimate" column to get average salary
- Clean Company Name 
- Parse out state from location
- Get age of company(how long the company has been around from the "Founded" column
- Amount of competitors
- Parse Job Description(coding languages required for job)
- Parse out Job Title into a few groups to make analysis easier


In [3]:
# Check for null values
df.isnull().sum()

Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64

In [4]:
# We see that we have no null values. 
# Next let's check for any values which represents null values
# In the salary column we have some values that = -1, let's get rid of those
df=df[df["Salary Estimate"]!="-1"]

df["Salary Estimate"].value_counts()

$49K-$113K (Glassdoor est.)                  6
$86K-$143K (Glassdoor est.)                  6
$54K-$115K (Glassdoor est.)                  6
$21-$34 Per Hour(Glassdoor est.)             6
$74K-$124K (Glassdoor est.)                  5
$76K-$142K (Glassdoor est.)                  5
$107K-$173K (Glassdoor est.)                 5
$81K-$167K (Glassdoor est.)                  5
$68K-$139K (Glassdoor est.)                  4
$63K-$105K (Glassdoor est.)                  4
$82K-$132K (Glassdoor est.)                  4
$61K-$109K (Glassdoor est.)                  4
$110K-$175K (Glassdoor est.)                 4
$40K-$68K (Glassdoor est.)                   4
$56K-$95K (Glassdoor est.)                   4
$49K-$97K (Glassdoor est.)                   4
$108K-$171K (Glassdoor est.)                 4
$44K-$78K (Glassdoor est.)                   4
$69K-$127K (Glassdoor est.)                  4
$42K-$86K (Glassdoor est.)                   4
$56K-$97K (Glassdoor est.)                   4
$39K-$66K (Gl

## Parsing salary

In [5]:
salary=df["Salary Estimate"].apply(lambda x:x.split("(")[0])
salary.head()

0     $53K-$91K 
1    $63K-$112K 
2     $80K-$90K 
3     $56K-$97K 
4    $86K-$143K 
Name: Salary Estimate, dtype: object

In [6]:
salary_minus_kd=salary.apply(lambda x:x.replace("$","").replace("K",""))
salary_minus_kd.head()

0     53-91 
1    63-112 
2     80-90 
3     56-97 
4    86-143 
Name: Salary Estimate, dtype: object

In [7]:
df["hourly"]=df["Salary Estimate"].apply(lambda x:1 if "per hour" in x.lower() else 0)
df["hourly"].value_counts()

0    718
1     24
Name: hourly, dtype: int64

In [8]:
df["Employer Provided"]=df["Salary Estimate"].apply(lambda x:1 if "employer provided" in x.lower() else 0)
df["Employer Provided"].value_counts()

0    725
1     17
Name: Employer Provided, dtype: int64

In [9]:
# Deleting Employer Provided and hourly in our new salary series

salary_upd=salary_minus_kd.apply(lambda x:x.lower().replace("per hour","").replace("employer provided",""))

# Delete the word salary

salary_new=salary_upd.apply(lambda x:x.lower().replace("salary","").replace(":",""))

In [10]:
df["min_salary"]=salary_new.apply(lambda x: int(x.split("-")[0]))
df["max_salary"]=salary_new.apply(lambda x: int(x.split("-")[1]))
df["avg_salary"]=(df.min_salary +df.max_salary)/2

In [11]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,hourly,Employer Provided,min_salary,max_salary,avg_salary
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5


We have added columns for min salary, max salary, and avg salary

## Cleaning up Company Name to get text only

In [19]:
df["Company_text"]=df["Company Name"].apply(lambda x:x.split("\n")[0])
df["Company_text"].head(2)

0                        Tecolote Research
1    University of Maryland Medical System
Name: Company_text, dtype: object

## Parsing states out of Location

In [20]:
df["States"]=df["Location"].apply(lambda x:x.split(",")[1])

In [21]:
df["States"].head(2)

0     NM
1     MD
Name: States, dtype: object

In [22]:
# Let's create a column to see if the location(state) is the same as the headquarter(state)

In [27]:
df["same_state"]=df.apply(lambda x:1 if x["Location"]==x["Headquarters"] else 0, axis=1)
df["same_state"].head()

0    0
1    0
2    1
3    1
4    1
Name: same_state, dtype: int64

## Company Age

In [68]:
df["Age"]=df["Founded"].apply(lambda x:2021-x if x!=-1 else x)
df["Age"].head(2)

0    48
1    37
Name: Age, dtype: int64

## Parsing job description based on coding languages

In [34]:
# Parsing of job description(common coding languages)
# Python, R studio, SQL, Spark,aws
df["python_yn"]=df["Job Description"].apply(lambda x:1 if "python" in x.lower() else 0)
df["rstudio_yn"]=df["Job Description"].apply(lambda x:1 if "rstudio" in x.lower() or "r-studio" in x.lower() else 0)
df["SQL_yn"]=df["Job Description"].apply(lambda x:1 if "sql" in x.lower() else 0)
df["spark_yn"]=df["Job Description"].apply(lambda x:1 if "spark" in x.lower() else 0)
df["aws_yn"]=df["Job Description"].apply(lambda x:1 if "aws" in x.lower() else 0)

In [35]:
df["python_yn"].value_counts()

1    392
0    350
Name: python_yn, dtype: int64

In [36]:
df["rstudio_yn"].value_counts()

0    741
1      1
Name: rstudio_yn, dtype: int64

In [37]:
df["SQL_yn"].value_counts()

1    380
0    362
Name: SQL_yn, dtype: int64

In [38]:
df["spark_yn"].value_counts()

0    575
1    167
Name: spark_yn, dtype: int64

In [39]:
df["aws_yn"].value_counts()

0    566
1    176
Name: aws_yn, dtype: int64

In [41]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'Employer Provided', 'min_salary', 'max_salary', 'avg_salary',
       'Company_text', 'States', 'same_state', 'Age', 'python_yn',
       'rstudio_yn', 'SQL_yn', 'spark_yn', 'aws_yn'],
      dtype='object')

In [42]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,avg_salary,Company_text,States,same_state,Age,python_yn,rstudio_yn,SQL_yn,spark_yn,aws_yn
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,72.0,Tecolote Research,NM,0,48,1,0,0,0,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,87.5,University of Maryland Medical System,MD,0,37,1,0,0,0,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,85.0,KnowBe4,FL,1,11,1,0,1,1,0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,76.5,PNNL,WA,1,56,1,0,0,0,0
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,114.5,Affinity Solutions,NY,1,23,1,0,1,0,0


## Simlifying Job Descriptions

In [43]:
def job_simplified(job):
    if "data scientist" in job.lower():
        return "data scientist"
    elif "analyst" in job.lower():
        return "analyst"
    elif "machine learning" in job.lower():
        return "mle"
    elif "data engineer" in job.lower():
        return "engineer"
    elif "manager" in job.lower():
        return "manager"
    elif "director" in job.lower():
        return "director"
    else:
        return "na"

## Measuring seniority

In [44]:
def seniority(title):
    if "senior" in title.lower() or "sr" in title.lower() or "snr" in title.lower() or "lead" in title.lower():
        return "senior"
    elif "junior" in title.lower() or "jr" in title.lower() or "intern" in title.lower():
        return "junior"
    else:
        return "na"

In [45]:
df["job_simp"]=df["Job Title"].apply(job_simplified)
df["seniority"]=df["Job Title"].apply(seniority)

In [46]:
df["job_simp"].value_counts()

data scientist    279
na                184
engineer          118
analyst           103
manager            22
mle                22
director           14
Name: job_simp, dtype: int64

In [47]:
df["seniority"].value_counts()

na        542
senior    195
junior      5
Name: seniority, dtype: int64

In [48]:
# Fixing Los Angeles in States column
df["States"].value_counts().tail()

 ID             2
 MN             2
 Los Angeles    1
 RI             1
 SC             1
Name: States, dtype: int64

In [50]:
df["States"]=df["States"].apply(lambda x:x.replace("Los Angeles","CA"))
df["States"].value_counts().tail()

 KS    3
 ID    2
 MN    2
 RI    1
 SC    1
Name: States, dtype: int64

## Job Description Length

In [52]:
df["job_desc"]=df["Job Description"].apply(lambda x:len(x))
df["job_desc"].head()

0    2536
1    4783
2    3461
3    3883
4    2728
Name: job_desc, dtype: int64

## Competitor Count

In [59]:
df["num_comp"]=df["Competitors"].apply(lambda x:len(x.split(",")) if x!="-1" else 0)
df["num_comp"].head()

0    0
1    0
2    0
3    3
4    3
Name: num_comp, dtype: int64

## Hourly Wage to Annual(Hourly wage times 2 roughly equals annual wage)

In [69]:
df["avg_salary"]=df.apply(lambda x:x["avg_salary"]*2 if x["hourly"]==1 else x["avg_salary"],axis=1)
df["min_salary"]=df.apply(lambda x:x["min_salary"]*2 if x["hourly"]==1 else x["min_salary"],axis=1)
df["max_salary"]=df.apply(lambda x:x["max_salary"]*2 if x["hourly"]==1 else x["max_salary"],axis=1)
df[["avg_salary","hourly"]]

,avg_salary,hourly
0,72.0,0
1,87.5,0
2,85.0,0
3,76.5,0
4,114.5,0
5,95.0,0
6,73.5,0
7,114.0,0
8,61.0,0
9,140.0,0


In [62]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,Age,python_yn,rstudio_yn,SQL_yn,spark_yn,aws_yn,job_simp,seniority,job_desc,num_comp
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,48,1,0,0,0,0,data scientist,na,2536,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,37,1,0,0,0,0,data scientist,na,4783,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,11,1,0,1,1,0,data scientist,na,3461,0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,56,1,0,0,0,0,data scientist,na,3883,3
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,23,1,0,1,0,0,data scientist,na,2728,3


In [65]:
# Clean up size
df["Size"]=df["Size"].apply(lambda x:x.replace("to","-"))
df["Size"].head()

0     501 - 1000 employees
1         10000+ employees
2     501 - 1000 employees
3    1001 - 5000 employees
4       51 - 200 employees
Name: Size, dtype: object

## Remove unnamed column

In [71]:
df_cleaned=df.drop("Unnamed: 0",axis=1)

In [72]:
df_cleaned.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Age,python_yn,rstudio_yn,SQL_yn,spark_yn,aws_yn,job_simp,seniority,job_desc,num_comp
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 - 1000 employees,1973,Company - Private,...,48,1,0,0,0,0,data scientist,na,2536,0
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,...,37,1,0,0,0,0,data scientist,na,4783,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 - 1000 employees,2010,Company - Private,...,11,1,0,1,1,0,data scientist,na,3461,0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 - 5000 employees,1965,Government,...,56,1,0,0,0,0,data scientist,na,3883,3
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 - 200 employees,1998,Company - Private,...,23,1,0,1,0,0,data scientist,na,2728,3


In [73]:
df_cleaned.to_csv('data_cleaned.csv')